In [ ]:
import boto3
import uuid

glue = boto3.client("glue", region_name="us-east-1")


In [ ]:
def wait_for_glue_job(glue, job_name, run_id):
    while True:
        resp = glue.get_job_run(
            JobName=job_name,
            RunId=run_id,
            PredecessorsIncluded=False
        )
        state = resp["JobRun"]["JobRunState"]

        if state in ("SUCCEEDED", "FAILED", "STOPPED", "TIMEOUT"):
            return state

        time.sleep(30)


In [ ]:
run = glue.start_job_run(
    JobName="DIM_SCD2_GENERIC",
    Arguments={
        "--run_dt": "2019-01-10",
        "--raw_base": "s3://my-bucket/raw",
        "--curated_base": "s3://my-bucket/curated",
        "--config_s3": "s3://my-bucket/config/dim_customer.json",
        "--run_id": f"backfill-{uuid.uuid4()}"
    }
)

In [ ]:
state = wait_for_glue_job(glue, "DIM_SCD2_GENERIC", run["JobRunId"])

if state != "SUCCEEDED":
    raise RuntimeError(f"Job failed with state {state}")